## Digital

This python notebook contains the digital software project, which has an ML model that recognizes the numbers your hands are holding up in real time

In [ ]:
import cv2 as cv
import numpy as np
import time
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import HandLandmarkerResult

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [3]:
detection_res = None

In [ ]:
# Supporting functions

def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global detection_res
    detection_res = result

    if result.hand_landmarks:
        print('Hands found!: ', result)


In [4]:
base_opt = python.BaseOptions(model_asset_path='models/hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_opt, num_hands=2, running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM, result_callback=print_result)

In [9]:
with HandLandmarker.create_from_options(options) as detector:
    cap = cv.VideoCapture(0)
    start_time = time.time()

    while cap.isOpened():
        success, frame = cap.read()
        frame = cv.flip(frame, 1)

        if not success:
            print("Can't receive frame :( exiting...")
            break

        timestamp_ms = int((time.time() - start_time)* 1000)

        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
        
        detector.detect_async(mp_image, timestamp_ms) 

        if detection_res and detection_res.hand_landmarks:
            for hand_landmarks in detection_res.hand_landmarks:
                for landmark in hand_landmarks:
                    x = int(landmark.x * frame.shape[1])
                    y = int(landmark.y * frame.shape[0])
                    cv.circle(frame, (x,y), 5, (0,255,0), -1)

        cv.imshow('frame',frame)

        if cv.waitKey(1) == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

I0000 00:00:1770615823.505137 11379969 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4 Pro
W0000 00:00:1770615823.508596 11379970 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1770615823.512306 11379973 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Hands found!:  HandLandmarkerResult(handedness=[[Category(index=0, score=0.9898092746734619, display_name='Right', category_name='Right')]], hand_landmarks=[[NormalizedLandmark(x=0.22755901515483856, y=0.8503071665763855, z=-1.972954493112411e-07, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.30469435453414917, y=0.8592144846916199, z=-0.0059099942445755005, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.3639989495277405, y=0.8401946425437927, z=-0.019184421747922897, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.4023260474205017, y=0.8114987015724182, z=-0.032687339931726456, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.42468446493148804, y=0.7770215272903442, z=-0.043753840029239655, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.38454923033714294, y=0.7408491969108582, z=-0.03695633262395859, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.4011707007884979, y=0.